In [43]:
from qdrant_client import QdrantClient
import os
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index import StorageContext, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI

In [44]:

qdrant_client = QdrantClient(
    os.getenv("QDRANT_ENDPOINT"), 
    prefer_grpc=True,
    api_key=os.getenv("QDRANT_API_KEY")
)
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
service_context = ServiceContext.from_defaults(llm=llm)
vector_store = QdrantVectorStore(client=qdrant_client, collection_name="city_query")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents,storage_context=storage_context, service_context=service_context)

In [60]:
query_engine = index.as_query_engine()

In [61]:
response = query_engine.query("what is the population of Houston? What is the population of New York?")
print(response)
print(response.get_formatted_sources())

The population of Houston is not provided in the given context information. The population of New York is also not provided in the given context information.
> Source (Doc id: e0bd2a3a-e8ec-4ad9-9b47-b5987e87bc2b): In 2009, Houston became the first U.S. city with a population over 1 million citizens to elect a ...

> Source (Doc id: e19e1232-4571-4b51-a257-4894a3546a66): In 2009, Houston became the first U.S. city with a population over 1 million citizens to elect a ...


In [55]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import get_response_synthesizer

retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=4,
)

response_synthesizer = get_response_synthesizer(verbose=True)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [59]:
response = query_engine.query("compare the population of Houston and New York")
print(response)
print(response.get_formatted_sources())

> Refine context: on their local economy, employment opportunitie...
Based on the given context, it is not possible to compare the population of Houston and New York as New York is not mentioned.
> Source (Doc id: e19e1232-4571-4b51-a257-4894a3546a66): In 2009, Houston became the first U.S. city with a population over 1 million citizens to elect a ...

> Source (Doc id: e0bd2a3a-e8ec-4ad9-9b47-b5987e87bc2b): In 2009, Houston became the first U.S. city with a population over 1 million citizens to elect a ...

> Source (Doc id: 1b2f8d17-e484-45fe-8138-3abaf86c1ae8): In 2010, mining (which consists almost entirely of exploration and production of oil and gas in H...

> Source (Doc id: 16f35567-8533-47f0-bbec-6abf790f5aae): In 2010, mining (which consists almost entirely of exploration and production of oil and gas in H...
